Pour remplir la mission 1 :

```python
import pandas as pd
import os
from sqlalchemy import create_engine
import pymysql

# Connexion à la base de données
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="303869",
                               pw="7852xawJBTjDhTc",
                               host="mysql-credit-scoring-ipssi.alwaysdata.net",
                               db="credit-scoring-ipssi_home-credit"))

dossier = os.getcwd()
fichiers_csv = [fichier for fichier in os.listdir(dossier) if fichier.endswith(".csv")]

for fichier in fichiers_csv:
    df = pd.read_csv(fichier)
    print(fichier, "lu")
    df.to_sql(con=engine, name=fichier[:-4], if_exists="replace", index=False, chunksize=1000)
    print(fichier, "importé")
```